In [41]:
from pint import UnitRegistry
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import ipywidgets as widgets
from ipywidgets import interactive

In [43]:
ureg = UnitRegistry()
# coat with 50 uL 5ug/mL mucin (200 AA peptide. 22kDa)
# wash and then 50 uL nanobody (25 kDa) (start 0.2 ug/mL og serie del med 2 10 gange og tilføj 0 (12 punkter i alt))
# wash and then 50 uL seconday 

print("Ag coat")
mass_mucin = ((50E-6 * ureg.liter) * ((5E-6 * ureg.gram) / (1E-3* ureg.liter)))
print(f"{mass_mucin:.0f~#P}")

moles_mucin = mass_mucin / (22E3 * ureg.g /ureg.mole)
print(f"{moles_mucin:.0f~#P}")

print()
print("start Ab")
mass_Ab = ((50E-6 * ureg.liter) * ((0.2E-6 * ureg.gram) / (1E-3* ureg.liter)))
print(f"{mass_Ab:.0f~#P}")

moles_Ab = mass_Ab / (16E3 * ureg.g /ureg.mole)
print(f"{moles_Ab:.0f~#P}")

Ag coat
250 ng
11 pmol

start Ab
10 ng
625 fmol


In [51]:
def rect_hyperbola(x, Bmax, Kd):
    return Bmax * x / (Kd + x)

default_kd = 1000e-9
default_coat = 10e-9
default_max_ab = 1e-9
default_ab_concentrations = np.array([default_max_ab/3**(x-1) for x in range(1,12)])
default_frac_bound = default_coat * default_ab_concentrations / (default_coat + default_kd + default_ab_concentrations)

def plot_hyperbola(kd, coat, max_ab):
    Kd = kd*1e-9
    Coat = coat*1e-9
    Max_ab = max_ab*1e-9
    ab_concentrations = np.array([Max_ab/3**(x-1) for x in range(1,12)])
    frac_bound = Coat * ab_concentrations / (Coat + Kd + ab_concentrations)
    
    params, cov = curve_fit(rect_hyperbola, ab_concentrations, frac_bound, p0=(Max_ab, Kd))
    #params, cov = curve_fit(rect_hyperbola, ab_concentrations, frac_bound)
    plt.plot(ab_concentrations, frac_bound, 'o', label='Data')
    plt.plot(default_ab_concentrations, default_frac_bound, 'x', label='default')
    all_xs = np.array(sorted([*list(default_ab_concentrations), *list(ab_concentrations)]))
    plt.plot(all_xs, rect_hyperbola(all_xs, *params), '-', label=f'Fit, kd:{params[1]:.3}')
    plt.xlabel('Nanobody Concentration (nM)')
    plt.ylabel('Fraction of Receptor Bound to Antigen')
    plt.semilogx()
    plt.vlines(Kd, min([*frac_bound, *default_frac_bound]), max([*frac_bound, *default_frac_bound]), 'r', linestyles="dashed")
    plt.vlines(params[1], min([*frac_bound, *default_frac_bound]), max([*frac_bound, *default_frac_bound]), linestyles="dashed")
    plt.title('Antibody-Antigen Binding Saturation ELISA')
    plt.legend()
    plt.show()

widgets.interact(plot_hyperbola,
                 kd=widgets.FloatSlider(min=1, max=10000, step=1, value=default_kd*1e9, description="Kd [nM]"),
                 coat=widgets.FloatSlider(min=0.01, max=100000, step=0.01, value=default_coat*1e12, description="antigen [pM]"),
                 max_ab=widgets.FloatSlider(min=1, max=10000000, step=1, value=default_max_ab*1e12, description="Max AB conc [pM]")
                )

interactive(children=(FloatSlider(value=1000.0, description='Kd [nM]', max=10000.0, min=1.0, step=1.0), FloatS…

<function __main__.plot_hyperbola(kd, coat, max_ab)>